<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/3_make_errors/e4_make_errors_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install kiwipiepy
# !pip install nltk
# !pip install
# !pip install selenium

#코랩에서 구글드라이버 연동
# !apt-get update
# !apt install chromium-chromedriver

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Mounted at /content/drive


**아이디어 기록**

1. Critical

2. Major
    - Mis-Translation
        - "ailen" -> 외계인으로 처리

# 0.Data Load

In [4]:
import pandas as pd
import numpy as np
import json
import os
import re
import copy
from tqdm.notebook import tqdm

if os.path.exists(r"G:\내 드라이브\KB인턴관련"):
    gdrive_path = r"G:\내 드라이브\KB인턴관련"
else:
    gdrive_path = "/content/drive/MyDrive/KB인턴관련"

In [5]:
df = pd.read_csv(os.path.join(gdrive_path, "preprocessing/fin.csv"), encoding="euc-kr", index_col=0)
df.head()

,category,lawName,ENG,KOR
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발·운영의 지원 및 개성공업지구에 투자하거나 출입·체류하는...
2,헌법,개성공업지구 지원에 관한 법률,"The term ""Gaeseong Industrial Complex"" or ""GIC...","""개성공업지구""란 남한과 북한 사이의 합의에 따라서 북한의 개성지역 일대에서 개발·..."
3,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC developer"" means a South Korean ...","""개성공업지구 개발업자""란 「남북교류협력에 관한 법률」에 따른 협력사업의 승인을 받..."
4,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC management organ"" means a corpor...","""개성공업지구 관리기관""이란 개성공업지구의 관리·운영을 위하여 북한의 「개성공업지구..."
5,헌법,개성공업지구 지원에 관한 법률,"The term ""local enterprise of the GIC"" means a...","""개성공업지구 현지기업""이란 남한주민이 「남북교류협력에 관한 법률」에 따른 협력사업..."


In [324]:
df.info(      )

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280765 entries, 0 to 585940
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   category  280765 non-null  object
 1   lawName   280765 non-null  object
 2   ENG       280765 non-null  object
 3   KOR       280765 non-null  object
dtypes: object(4)
memory usage: 18.8+ MB


# 1.Data Fiiltering

In [6]:
keyword_idx = df.loc[df.lawName.str.contains(r"은행|금융|보험|예금|적금|대출|외환|통화|화폐|환율|예대")].index

In [7]:
tgk_idx = df.loc[df.category=="통화ㆍ국채ㆍ금융"].index

In [8]:
print(f"키워드: {len(keyword_idx)}\t통화/국채/금융: {len(tgk_idx)}")
print(f"키워드-통국금: {len(list(set(keyword_idx)-set(tgk_idx)))}")
print(f"통국금-키워드: {len(list(set(tgk_idx)-set(keyword_idx)))}")
print(f"전체:  {len(list(set(keyword_idx).union(set(tgk_idx))))}")

키워드: 16712	통화/국채/금융: 16004
키워드-통국금: 4681
통국금-키워드: 3973
전체:  20685


In [430]:
df_samp = df.loc[list(set(keyword_idx).union(set(tgk_idx)))].drop(index=[403163], axis=0)
# df_samp = df.copy(deep=True)

In [431]:
df_samp["curHyp"] = df_samp.KOR.values
df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
df_samp["tagLog"] = [dict() for _ in range(df_samp.shape[0])]
df_samp["errorLog"] = [dict() for _ in range(df_samp.shape[0])]
# df_samp.iloc[:, 2:4]

**주요 함수**

In [548]:
## 주요한 함수
def remake_df():
    result = df.loc[list(set(keyword_idx).union(set(tgk_idx)))]
    result["curHyp"] = result.KOR.values
    result["errors"] = [[] for _ in range(result.shape[0])]
    result["tag_log"] = [dict() for _ in range(result.shape[0])]
    return result.drop(index=[403163], axis=0)

def random_df(dfs, ratio, seed):
    np.random.seed(seed)
    return dfs.loc[np.random.choice(dfs.index.to_list(), int(dfs.shape[0]*ratio), False), :]

def random_index(srs, ratio, seed):
    np.random.seed(seed)
    return np.random.choice(srs.index, int(srs.shape[0]*ratio), False)

def copy_df(data, col=["errors", "errorLog", "errorLog"]):
    result = data.drop(columns=col, axis=1).copy(deep=True)
    # print(id(result), id(data))
    for c in col:
        result[c] = data[c].apply(lambda x: copy.deepcopy(x)).copy(deep=True)
    return result

def count_tag(tokens, tags=None):
    token_dict = dict()
    for token in tokens:
        if tags!=None and token.tag not in tags: continue
        try:
            token_dict[token.tag].append(token.form)
        except:
            token_dict[token.tag] = [token.form]

    return token_dict

def find_tokens(tokens, tag):
    result_tokens = []
    for token in tokens:
        if token.tag == tag:
            result_tokens.append((token.form, token.start, token.end))
    return result_tokens

def print_df(df, cols, idx=0, end=0):
    print(df.shape[0])
    if end==0:
        values = df.index[idx:]
    else:
        values = df.index[idx:end]
    for i in values:
        print(f"_________________ {i} _________________")
        for col in cols:
            print(df.loc[i, col])

# result.loc[i, "errors"].append({"severity":"major", "start_idx":start, "text":del_unit, "type":"omission"})
def input_errors(df, index, severity, start_idx, text, type):
    df.loc[index, "errors"].append({"severity":severity, "start_idx":start_idx, "text":text, "type":type})

def convert_df(df, diff_df):
    cdf = copy_df(df)
    for i in diff_df.index:
        cdf.loc[i, : ] = diff_df.loc[i, : ]
    return cdf

def dup_drop():
    for k in dindex_dict.keys():
        dindex_dict[k] = list(set(dindex_dict[k]))

def check_idx(df):
    dup_drop()
    for k in dindex_dict.keys():
        print(f"{k}: {len(dindex_dict[k])}")
    print("############################################")
    print("omission: ", df.loc[df.errorLog.apply(lambda x: "omission" in x.keys())].shape[0])
    print("addition: ", df.loc[df.errorLog.apply(lambda x: "addition" in x.keys())].shape[0])
    print("mis-translation: ", df.loc[df.errorLog.apply(lambda x: "mis-translation" in x.keys())].shape[0])
    print("over-translation: ", df.loc[df.errorLog.apply(lambda x: "over-translation" in x.keys())].shape[0])
    print("under-translation: ", df.loc[df.errorLog.apply(lambda x: "under-translation" in x.keys())].shape[0])
    print("grammer: ", df.loc[df.errorLog.apply(lambda x: "grammer" in x.keys())].shape[0])

# 2.품사 태깅

In [12]:
from kiwipiepy import Kiwi

kiwi = Kiwi(num_workers=0, model_type='sbg')
kiwi_tokens = df_samp.KOR.apply(lambda x: kiwi.tokenize(x))
kiwi_tokens

547121    [(예술인, NNG, 0, 3), ([, SSO, 3, 1), (「, SSO, 4,...
547123    [(노무, NNG, 0, 2), (제공자, NNG, 2, 3), ([, SSO, 5...
547124    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547125    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547126    [(예술, NNG, 0, 2), (이, VCP, 2, 1), (ᆫ, ETM, 2, ...
                                ...                        
78005     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
78006     [(제, XPN, 0, 1), (1, SN, 1, 1), (항, NNG, 2, 1)...
78007     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
406361    [(제, XPN, 0, 1), (3, SN, 1, 1), (항, NNG, 2, 1)...
406362    [(조합, NNG, 0, 2), (이, JKS, 2, 1), (제, XPN, 4, ...
Name: KOR, Length: 20684, dtype: object

In [13]:
count_tokens = kiwi_tokens.apply(count_tag)
count_tokens

547121    {'NNG': ['예술인', '항', '단서', '단기', '예술인', '이하', ...
547123    {'NNG': ['노무', '제공자', '항', '단서', '단기', '노무', '...
547124    {'NNG': ['법', '항', '대통령령', '사유', '해당', '경우', '...
547125    {'NNG': ['법', '항', '법', '항', '항', '준용', '경우', ...
547126    {'NNG': ['예술', '다음', '호의', '해당', '경우', '법', '항...
                                ...                        
78005     {'NNG': ['기획', '재정부', '장관', '국채', '발행', '관리', ...
78006     {'XPN': ['제'], 'SN': ['1'], 'NNG': ['항', '국채',...
78007     {'NNG': ['기획', '재정부', '장관', '국채', '시장', '정책', ...
406361    {'XPN': ['제', '제', '제', '제', '제'], 'SN': ['3',...
406362    {'NNG': ['조합', '항', '규정', '복지', '사업', '실시', '경...
Name: KOR, Length: 20684, dtype: object

In [14]:
nng_dict = dict()
for tokens in count_tokens.apply(lambda x: x["NNG"]):
    for t in tokens:
        try:
            nng_dict[t] += 1
        except:
            nng_dict[t] = 1
len(nng_dict.keys())

3703

In [15]:
tag = "NNG"
nouns =list(set(count_tokens.apply(lambda x: x[tag] if tag in x.keys() else []).sum()))
len(nouns)

547121    [(고용보험법, 5, 10)]
547123    [(고용보험법, 7, 12)]
547124                  []
547125                  []
547126                  []
                ...       
78005     [(한국은행, 56, 60)]
78006                   []
78007                   []
406361                  []
406362                  []
Name: KOR, Length: 20684, dtype: object

# 3.Error 생성

In [555]:
dindex_dict = {"omission":[], "addition":[], "mis-translation":[], "over-translation":[], "under-translation":[], "grammer": []}

## Omission

### Omission - Critical

In [556]:
cr_om_index = random_index(df_samp.curHyp, 0.02, 10)
# df_samp.loc[cr_om_index]

In [557]:
def critical_omission(df, seed, ratio):
    df_t = copy_df(df)
    cr_om_index = random_index(df_t.curHyp, ratio, seed)
    dindex_dict["omission"]+=list(cr_om_index)
    result = df_t.loc[cr_om_index]
    random_ratio = [3/5, 2/3, 5/8, 5/7]
    np.random.seed(seed)
    split_sentences = result.curHyp.apply(lambda x: x.split(" "))
    len_element = split_sentences.apply(len)
    crop_element = len_element.apply(lambda x: int(x*(np.random.choice(random_ratio, 1))))
    start_idx = (len_element-crop_element).apply(lambda x: np.random.choice([i for i in range(x)], 1)[0])
    end_idx = start_idx+crop_element

    for i in cr_om_index:
        reduce_sentence = (" ".join([split_sentences[i][k] for k in range(start_idx[i], end_idx[i]+1)])).strip()
        cur_hyp = result.loc[i, "curHyp"]
        sidx = cur_hyp.find(reduce_sentence)

        conv_hyp = " ".join([split_sentences[i][j] for j in range(len_element[i]) if (start_idx[i] > j) or (end_idx[i] < j)])
        result.loc[i, "curHyp"] = conv_hyp
        result.loc[i, "tagLog"]["omission"] = sidx
        input_errors(result, i, "critical", sidx, reduce_sentence, "omission")
        result.loc[i, "errorLog"]["omission"] = {"start_idx": sidx, "text": reduce_sentence}

    del split_sentences, len_element, crop_element, start_idx, end_idx
    return convert_df(df, result)

df_crom = critical_omission(df_samp, 10, 0.02)

In [558]:
df_crom.loc[cr_om_index].shape

(413, 8)

In [530]:
# df_crom.loc[cr_om_index]

In [559]:
check_idx(df_crom)

omission: 413
addition: 0
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  413
addition:  0
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


### Omission - Major

In [532]:
np.random.seed(20)
nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, tag)).drop(index=dindex_dict["omission"], axis=0)
np.random.seed(20)
change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 3]
chv_index = random_index(ch6, 0.65, 20)

In [560]:
def major_omission(df_local, seed, tag, slen, ratio):
    df_t = copy_df(df_local)
    np.random.seed(seed)
    nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, tag)).drop(index=dindex_dict["omission"], axis=0)
    np.random.seed(seed)
    change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
    ch6 = change_val[change_val.apply(lambda x: len(x[0])) > slen]
    chv_index = random_index(ch6, ratio, seed)
    # drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
    ch_srs = ch6.loc[chv_index]
    result = df_t.loc[chv_index]

    for i in chv_index:
        del_word, start, end = ch_srs.loc[i]
        cur_hyp = result.loc[i, "curHyp"]

        forward = cur_hyp[:start]
        backward = cur_hyp[start:]
        split_back = backward.split(" ")
        del_unit = split_back[0]
        back_mod = " ".join(split_back[1:])

        try:
            if start==0:
                conv_hyp = (forward+back_mod).strip(" ")
            elif cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' or cur_hyp[end]=="」" or cur_hyp[end]=='\"' :
                pattern = f'\"[^\"]*{del_word}[^\"]*\"|「[^」]*{del_word}[^」]*」'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            elif "(" in del_unit:
                pattern = f'{del_word}[^\)]+\)'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            else:
                conv_hyp = (forward+back_mod).strip(" ")
            result.loc[i, "tagLog"]["omission"] = start
            dindex_dict["omission"]+=[i]
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "major", start, del_unit, "omission")
            result.loc[i, "errorLog"]["omission"] = {"start_idx": start, "text": del_unit}
        except:
            pass

    return convert_df(df_local, result)

df_maom = major_omission(df_crom, 20, "NNP", 3, 0.65)
# df_mo = major_omission(df_maom, 36, "NNG", 2, 0.55)

In [561]:
df_maom.loc[df_maom.errors.apply(len)==1].shape[0]

1967

In [562]:
# df_maom.loc[df_maom.errors.apply(len)==1]

In [563]:
check_idx(df_maom)

omission: 1967
addition: 0
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  1967
addition:  0
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


In [564]:
df_mo = major_omission(df_maom, 36, "NNG", 2, 0.55)

In [565]:
check_idx(df_mo)

omission: 3049
addition: 0
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  3049
addition:  0
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


In [539]:
# df_mo.loc[df_mo.errors.apply(len)==1]

### Omission - Neutral

In [566]:
sp_list = [".", ";", ":"]
np.random.shuffle(sp_list)
sp_list

['.', ':', ';']

In [567]:
test = "안녕하세요; 반갑습니다."
a = test.find(".")
test[:a] + test[a+1:]

'안녕하세요; 반갑습니다'

In [568]:
def omission_nt(df, ratio, seed):    # 0.01, 111
    df_t = copy_df(df)
    np.random.seed(seed)
    on_index = random_index(df_t.drop(index=dindex_dict["omission"]), ratio, seed)
    on_index = df_t.loc[on_index][df_t.loc[on_index].curHyp.str.contains(r"\.|\;|\:")].index
    result = df_t.loc[on_index]
    dindex_dict["omission"]+=list(on_index)

    sp_list = [".", ";", ":"]
    for i in on_index:
        np.random.shuffle(sp_list)
        for s in sp_list:
            cur_hyp = result.loc[i, "curHyp"]
            s_loc = cur_hyp.find(s)
            if s_loc==-1:
                continue
            conv_hyp = cur_hyp[:s_loc]+cur_hyp[s_loc+1:]
            result.loc[i, "tagLog"]["omission"] = s_loc
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "neutral", s_loc, s, "omission")
            result.loc[i, "errorLog"]["omission"] = {"start_idx": s_loc, "text": s}
            break
    return convert_df(df, result)

df_on = omission_nt(df_mo, 0.01, 111)

In [569]:
# df_on.loc[df_on.errors.apply(lambda x: x[0]["severity"]=="neutral" if len(x)!=0 else False)]

In [570]:
check_idx(df_on)

omission: 3190
addition: 0
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  3190
addition:  0
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


## Addition

###Addition-Major

In [545]:
def major_addition(df, ratio, seed):
    np.random.seed(seed)
    df_t=copy_df(df)

    ma_ad_index = random_index(df_t, ratio, seed)
    nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])
    nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
    drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
    nnpg_info.drop(index=drop_index, axis=0, inplace=True)
    ma_ad_index = list(set(ma_ad_index)-set(drop_index))
    np.random.seed(seed)
    add_position = nnpg_info.apply(lambda x: x[np.random.choice(range(0, len(x)))])
    np.random.seed(seed)
    add_words = nouns_lists.apply(lambda x: np.random.choice(list(set(nouns)-set(x))))
    result = df_t.loc[ma_ad_index]

    dindex_dict["addition"]+=ma_ad_index

    josa_list = ["의", "에 의한", "에", "만", "만의"]
    for i in ma_ad_index:
        cur_hyp = df.loc[i, "curHyp"]
        josa_add = np.random.choice(josa_list)
        word = add_position[i]
        start = cur_hyp.find(word)
        # if word not in cur_hyp:
        #     continue
        add_word = add_words[i]+josa_add
        while start>=0 and cur_hyp[start]!=" ":
            start-=1
        conv_hyp = cur_hyp[:start+1].strip()+" "+f"{add_word}"+" "+cur_hyp[start+1:].strip()
        result.loc[i, "tagLog"]["addition"] = add_word
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "major", start+1, add_word, "addition")

        err_log = copy.deepcopy(result.loc[i, "errorLog"])
        loop_keys = err_log.keys()
        for k in loop_keys:
            pre_idx = err_log[k]["start_idx"]
            if pre_idx > start+1:
                result.loc[i, "errorLog"][k] = pre_idx + (len(add_word)+1)

        result.loc[i, "errorLog"]["addition"] = {"start_idx": (start+1), "text": add_word}

    return convert_df(df, result)

df_maad = major_addition(df_on, 0.22, 55)

In [546]:
# df_maad.loc[df_maad.errors.apply(len)>1]

In [550]:
set(list(df_maad.loc[df_maad.errorLog.apply(lambda x: "addition" in x.keys())].index))

4469

In [554]:
who = list(set(dindex_dict["addition"]) - set(list(df_maad.loc[df_maad.errorLog.apply(lambda x: "addition" in x.keys())].index)))
df_maad.loc[who]

,category,lawName,ENG,KOR,curHyp,errors,tagLog,errorLog
207882,해운,어선원 및 어선 재해보상보험법,"In compelling the payment under paragraph (2),...",중앙회는 제2항에 따라 납부를 독촉하는 경우에는 독촉장을 발급하여야 한다. 이 경우...,납부기한은 10일 이상의 여유가 있도록 하여야 한다.,"[{'severity': 'critical', 'start_idx': 0, 'tex...",{'omission': 0},"{'omission': {'start_idx': 0, 'text': '중앙회는 제2..."
86028,통화ㆍ국채ㆍ금융,한국수출입은행법,The Minister of Strategy and Finance may entru...,기획재정부장관은 제1항에 따른 검사를 금융감독원장에게 위탁할 수 있다.,수 있다.,"[{'severity': 'critical', 'start_idx': 0, 'tex...",{'omission': 0},"{'omission': {'start_idx': 0, 'text': '기획재정부장관..."
405009,통화ㆍ국채ㆍ금융,신용정보의 이용 및 보호에 관한 법률 시행령,"""Risk management system prescribed by Presiden...","법 제26조의4제4항 각 호 외의 부분에서 ""대통령령으로 정하는 위험관리체계""란 다...",법 제26조의4제4항 위험관리체계를 말한다.,"[{'severity': 'critical', 'start_idx': 12, 'te...",{'omission': 12},"{'omission': {'start_idx': 12, 'text': '각 호 외의..."
408081,통화ㆍ국채ㆍ금융,예금자보호법 시행령,"Where the Corporation moves its main office, b...",동일한 등기소의 관할구역안에서 주된 사무소·지사무소 또는 출장소를 이전한 때에는 2...,동일한 등기소의 관할구역안에서 주된 사무소·지사무소 또는 이전한 때에는 2주이내에 ...,"[{'severity': 'major', 'start_idx': 32, 'text'...",{'omission': 32},"{'omission': {'start_idx': 32, 'text': '출장소를'}}"
395833,통화ㆍ국채ㆍ금융,금융산업의 구조개선에 관한 법률,The applicant institution shall not receive su...,신청기관이 정부 또는 예금보험공사로부터 제12조 또는 「예금자보호법」 제38조에 따...,지원을 받지 아니할 것,"[{'severity': 'critical', 'start_idx': 0, 'tex...",{'omission': 0},"{'omission': {'start_idx': 0, 'text': '신청기관이 정..."
...,...,...,...,...,...,...,...,...
394206,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"Where the Minister of Trade, Industry and Ener...",산업통상자원부장관은 제4항이나 제5항에 따라 경제자유구역을 지정한 경우에는 대통령령...,산업통상자원부장관은 제4항이나 제5항에 따라 경제자유구역을 지정한 경우에는 대통령령...,"[{'severity': 'major', 'start_idx': 125, 'text...",{'omission': 125},"{'omission': {'start_idx': 125, 'text': '「토지이용..."
413668,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,A person who trades financial investment instr...,제70조를 위반하여 투자자로부터 예탁받은 재산으로 금융투자상품의 매매를 한 자,제70조를 위반하여 예탁받은 재산으로 금융투자상품의 매매를 한 자,"[{'severity': 'major', 'start_idx': 11, 'text'...",{'omission': 11},"{'omission': {'start_idx': 11, 'text': '투자자로부터'}}"
421862,통화ㆍ국채ㆍ금융,한국은행법,The Governor shall submit the statement of acc...,총재는 회계연도가 끝난 후 2개월 이내에 외부감사를 받은 해당 연도 결산서(재무제표...,총재는 회계연도가 끝난 후 기획재정부장관에게 각각 제출하여야 한다.,"[{'severity': 'critical', 'start_idx': 15, 'te...",{'omission': 15},"{'omission': {'start_idx': 15, 'text': '2개월 이내..."
417779,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률 시행령,"""Case prescribed by Presidential Decree"" in Ar...","법 제249조의9제1항제6호에서 ""대통령령으로 정하는 경우""란 다음 각 호의 어느 ...",법 경우를 말한다.,"[{'severity': 'critical', 'start_idx': 2, 'tex...",{'omission': 2},"{'omission': {'start_idx': 2, 'text': '제249조의9..."


### Addition-Minor

In [420]:
def ma_iter(df, ratio, seed):   #0.22 234
    df_t = copy_df(df)
    np.random.seed(seed)
    am_index = random_index(df_t.drop(index=dindex_dict["addition"]), ratio, seed)
    result = df_t.loc[am_index]
    dindex_dict["addition"]+=list(am_index)

    for i in am_index:
        cur_hyp = result.loc[i, "curHyp"]
        sch = cur_hyp.split(" ")
        rand_idx = np.random.choice([i for i in range(len(sch))])
        random_dup = sch[rand_idx]
        conv_hyp=" ".join(sch[:rand_idx] + [random_dup] +sch[rand_idx:])
        start = conv_hyp.find(random_dup) + len(random_dup) + 1

        result.loc[i, "tagLog"]["addition"] = random_dup
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "minor", start, random_dup, "addition")

        try:
            o_index = result.loc[i, "tagLog"]["omission"]
            if o_index > start:
                result.loc[i, "tagLog"]["omission"] += (len(random_dup)+1)
        except:
            pass

    return convert_df(df, result)
df_miter = ma_iter(df_maad, 0.2, 234)

In [421]:
# am_index = random_index(df_maad.drop(index=dindex_dict["addition"]), 0.22, 234)
# df_miter.loc[am_index]

In [422]:
check_idx(df_miter)

omission: 3190
addition: 7712
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  3190
addition:  7712
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  0


## Mis-Translation

### MT - Minor

In [423]:
def hv_mistrans(df, ratio, seed):   # 0.3 789
    df_t = copy_df(df)
    temp = df_t.drop(index=dindex_dict["mis-translation"]).loc[df_t.curHyp.str.contains(r'하여야 한다')]
    can_index = random_index(temp, ratio, seed)
    result = df_t.loc[can_index]
    ch_words = ['해야만 한다', '해야 할 것이다']
    dindex_dict["mis-translation"] += list(can_index)

    for i in can_index:
        np.random.shuffle(ch_words)
        ch_word = ch_words[0]
        cur_hyp = result.loc[i, "curHyp"]
        start = cur_hyp.find('하여야 한다')
        conv_hyp = cur_hyp[:start] + ch_word + cur_hyp[start+6:]

        result.loc[i, "tagLog"]["mis-translation"] = ch_word
        result.loc[i, 'curHyp'] = conv_hyp
        input_errors(result, i, 'minor', start, ch_word, 'mis-translation')

        try:
            o_index = result.loc[i, "tagLog"]["omission"]
            if o_index > start:
                if ch_word == "해야 할 것이다":
                    result.loc[i, "tagLog"]["omission"] += 2
        except:
            pass
    return convert_df(df, result)

df_can = hv_mistrans(df_miter, 0.3, 789)

In [424]:
# df_can.loc[df_can.curHyp.str.contains('해야만 한다|해야 할 것이다')]

In [425]:
temp = df_can.drop(index=dindex_dict["mis-translation"])
# ddo_idx = temp.loc[temp.curHyp.str.contains(r"\s또는\s")]
# mit_idx = temp.loc[temp.curHyp.str.contains(r"\s및\s")].index
# nae_idx = temp.loc[temp.curHyp.str.contains(r"\s내지\s")].index

In [426]:
def ddo_trans(df, ratio, seed, val):
    df_t = copy_df(df)
    conv_list = ["내지", "및", "또는"]
    conv_list.remove(val)
    temp = df_t.drop(index=dindex_dict["mis-translation"])
    ival = val+" "
    ddo_idx = random_index(temp.loc[temp.curHyp.str.contains(ival)], ratio, seed)
    dindex_dict["mis-translation"]+=list(ddo_idx)

    result = df_t.loc[ddo_idx]
    for i in ddo_idx:
        np.random.shuffle(conv_list)
        conv_word = conv_list[0]
        cur_hyp = result.loc[i, "curHyp"]
        start = cur_hyp.find(ival)
        conv_hyp = cur_hyp[:start] + conv_word + cur_hyp[start+len(val):]

        result.loc[i, "tagLog"]["mis-translation"] = conv_word
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, 'minor', start, conv_word, 'mis-translation')

        try:
            o_index=result.loc[i, "tagLog"]["omission"]
            if o_index > start:
                if len(val) > len(conv_word):
                    result.loc[i, "tagLog"]["omission"]-=(len(val)-len(conv_word))
                elif len(val) < len(conv_word):
                    result.loc[i, "tagLog"]["omission"]+=(len(conv_word)-len(val))
        except:
            pass
    return convert_df(df, result)

df_ddo = ddo_trans(df_can, 0.2, 555, "또는")
df_nae = ddo_trans(df_ddo, 0.25, 444, "내지")
df_mit = ddo_trans(df_nae, 0.3, 333, "및")

In [427]:
check_idx(df_mit)

omission: 3190
addition: 7712
mis-translation: 3272
over-translation: 0
under-translation: 0
grammer: 0
############################################
omission:  3190
addition:  7712
mis-translation:  3272
over-translation:  0
under-translation:  0
grammer:  0


In [429]:
# def hada_trans(df, ratio, seed):
#     df_t = copy_df(df)

#     df_mit.loc[df_mit.curHyp.str.contains("한다")]
# 나중에 해야 인덱스 맞출 수 있을 듯

,category,lawName,ENG,KOR,curHyp,errors,tagLog
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...","제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술인""이라 한다)과 같은...","[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'addition': '기분만의'}"
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""...","제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""단기노무제공자""라 한다)와...","[{'severity': 'major', 'start_idx': 0, 'text':...",{'omission': 0}
547124,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,"""Where any ground prescribed by Presidential D...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어...","[{'severity': 'minor', 'start_idx': 48, 'text'...",{'addition': '어느'}
547125,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,The standard remuneration under Article 3 (2) ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...,[],{}
547126,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where an artist falls under any of the followi...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...,이득에 의한 예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8...,"[{'severity': 'major', 'start_idx': 0, 'text':...",{'addition': '이득에 의한'}
...,...,...,...,...,...,...,...
78001,통화ㆍ국채ㆍ금융,국채법,The right to receive payments of principal and...,국채의 원금 및 이자를 받을 권리는 5년간 행사하지 아니하면 시효(시효)의 완성으로...,국채의 원금 및 이자를 받을 권리는 5년간 행사하지 아니하면 시효(시효)의 완성으로...,[],{}
78003,통화ㆍ국채ㆍ금융,국채법,Article 36 (4) and (5) of the Management of th...,제1항에 따라 한국은행이 처리하는 국채에 관한 사무 중 국채 발행에 따라 수입되는 ...,제1항에 따라 한국은행이 처리하는 국채에 관한 사무 중 국채 발행에 따라 수입되는 ...,"[{'severity': 'major', 'start_idx': 96, 'text'...",{'addition': '신주에'}
78004,통화ㆍ국채ㆍ금융,국채법,The Governor of the Bank of Korea shall file a...,한국은행 총재는 제15조에 따른 국채에 관한 사무의 처리 내용을 기획재정부령으로 정...,총재는 제15조에 따른 국채에 관한 사무의 처리 내용을 기획재정부령으로 정하는 바에...,"[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'mis-translation': '해야만 한다'}"
78005,통화ㆍ국채ㆍ금융,국채법,Where necessary to properly issue and manage S...,기획재정부장관은 국채의 발행과 관리를 적절하게 하기 위하여 필요한 경우에는 관계 중...,기획재정부장관은 국채의 발행과 관리를 적절하게 하기 위하여 필요한 경우에는 관계 관...,"[{'severity': 'minor', 'start_idx': 45, 'text'...",{'addition': '관계'}


In [385]:
# df_mit.loc[df_mit.curHyp.str.contains("및") & df_mit.tagLog.apply(lambda x: set(x.keys())=={"omission", "mis-translation"})]

### MT - Major

In [386]:
test = '법 제16조의10제3항 단서에서 "1개월간 소정근로시간이 60시간 미만인 사람 등 대통령령으로 정하는 근로자"란 「고용보험법 시행령」 제3조제1항에 따른 고용보험 적용 제외 근로자 및 같은 영 제3조의3제2호 각 목에 해당하는 사람으로서 고용보험 가입을 신청하지 않은 사람을 말한다.'
compiler = re.compile(r"제[\d]+조[\s|\w]*제[\d]+항")
for f in compiler.finditer(test): print(f)

<re.Match object; span=(2, 12), match='제16조의10제3항'>
<re.Match object; span=(75, 81), match='제3조제1항'>


In [387]:
re.split(r"[^\d]+",'제16조의10제3항')

['', '16', '10', '3', '']

In [389]:
df_mit.loc[df_mit.curHyp.str.contains(r"[\d]+조[\s|\w]*제[\d]+항")]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...","제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술인""이라 한다)과 같은...","[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'addition': '기분만의'}"
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""...","제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""단기노무제공자""라 한다)와...","[{'severity': 'major', 'start_idx': 0, 'text':...",{'omission': 0}
547124,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,"""Where any ground prescribed by Presidential D...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어...","[{'severity': 'minor', 'start_idx': 45, 'text'...",{'addition': '어느'}
547125,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,The standard remuneration under Article 3 (2) ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...,[],{}
547126,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where an artist falls under any of the followi...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...,이득에 의한 예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8...,"[{'severity': 'major', 'start_idx': 0, 'text':...",{'addition': '이득에 의한'}
...,...,...,...,...,...,...,...
406311,통화ㆍ국채ㆍ금융,신용협동조합법 시행령,A union which has an auditor as a standing mem...,제14조제5항에 따라 감사를 상임으로 하는 조합은 같은 항에 따른 요건에 해당하게 ...,제14조제5항에 따라 감사를 상임으로 하는 조합은 같은 항에 따른 요건에 해당하게 ...,"[{'severity': 'major', 'start_idx': 54, 'text'...",{'addition': '청취만'}
406314,통화ㆍ국채ㆍ금융,신용협동조합법 시행령,Any of the following persons shall be eligible...,법 제27조제6항 또는 제8항 및 이 영 제14조제1항 또는 제5항에 따라 상임이사...,법 제27조제6항 또는 제8항 및 이 영 제14조제1항 또는 제5항에 따라 상임이사...,[],{}
78003,통화ㆍ국채ㆍ금융,국채법,Article 36 (4) and (5) of the Management of th...,제1항에 따라 한국은행이 처리하는 국채에 관한 사무 중 국채 발행에 따라 수입되는 ...,제1항에 따라 한국은행이 처리하는 국채에 관한 사무 중 국채 발행에 따라 수입되는 ...,"[{'severity': 'major', 'start_idx': 96, 'text'...",{'addition': '신주에'}
78005,통화ㆍ국채ㆍ금융,국채법,Where necessary to properly issue and manage S...,기획재정부장관은 국채의 발행과 관리를 적절하게 하기 위하여 필요한 경우에는 관계 중...,기획재정부장관은 국채의 발행과 관리를 적절하게 하기 위하여 필요한 경우에는 관계 관...,"[{'severity': 'minor', 'start_idx': 42, 'text'...",{'addition': '관계'}


In [392]:
df_mit.loc[df_mit.errors.apply(len)>0].shape

(11526, 7)

## Over / Under Translation

- 준용하다 -> 적용하다 : 과잉번역
- 적용하다 -> 준용하다 : 과소번역

In [325]:
def jy_trans(df, ratio, seedp):


,category,lawName,ENG,KOR,curHyp,errors,tagLog
394179,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Paragraphs (2) and (3) shall apply mutatis mut...,경제자유구역기본계획의 변경에 관하여는 제2항 및 제3항을 준용한다.,경제자유구역기본계획의 변경에 관하여는 제2항 및 제3항을 준용한다.,[],{}
394247,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Provisions of Articles 4 through 6 shall apply...,제1항 및 제2항에 따른 변경의 절차 및 요건에 관하여는 제4조부터 제6조까지의 규...,제1항 및 제2항에 따른 변경의 절차 및 요건에 관하여는 제4조부터 제6조까지의 규...,[],{}
394294,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Article 7 (3) and (4) shall apply mutatis muta...,제1항에 따른 경제자유구역개발계획의 변경 절차에 대해서는 제7조제3항 및 제4항을 ...,제1항에 따른 경제자유구역개발계획의 변경 부족만 절차에 대해서는 제7조제3항 및 제...,"[{'severity': 'major', 'start_idx': 23, 'text'...",{'addition': '부족만'}
394312,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"Article 4 (1) through (5), (8), and (9) shall ...","제1항에 따른 경제자유구역의 지정 해제에 관하여는 제4조제1항부터 제5항까지, 제8...",제1항에 따른 알림만의 경제자유구역의 지정 해제에 관하여는 제4조제1항부터 제5항까...,"[{'severity': 'major', 'start_idx': 8, 'text':...","{'addition': '알림만의', 'mis-translation': '내지'}"
394347,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Where the Mayor/Do Governor revokes the design...,시·도지사는 제1항에 따라 개발사업시행자의 지정을 취소한 경우에는 새로운 개발사업시...,시·도지사는 제1항에 따라 개발사업시행자의 지정을 취소한 경우에는 새로운 개발사업시...,[],{}
...,...,...,...,...,...,...,...
194419,노동,산업재해보상보험법 시행령,The main clause of Article 107 (1) and paragra...,"소위원회의 운영에 관하여는 제107조제1항 본문, 같은 조 제2항·제4항 및 제5항...","소위원회의 운영에 관하여는 제107조제1항 본문, 같은 조 제2항·제4항 및 제5항...","[{'severity': 'major', 'start_idx': 66, 'text'...",{'omission': 66}
194464,노동,산업재해보상보험법 시행령,"Articles 21 through 85, 96 through 98, 101 thr...",법 제123조에 따른 현장실습생에게 보험급여를 지급하는 경우 등에 관하여는 제21조...,법 제123조에 따른 현장실습생에게 보험급여를 지급하는 경우 등에 관하여는 제21조...,[],{}
194470,노동,산업재해보상보험법 시행령,"Articles 27, 28, 30 through 35, and 36 shall a...",법 제124조제2항에 따른 중·소기업 사업주에 대한 업무상의 재해의 인정 범위에 관...,법 제124조제2항에 따른 중·소기업 사업주에 대한 업무상의 재해의 인정 범위에 관...,"[{'severity': 'major', 'start_idx': 95, 'text'...","{'omission': 96, 'mis-translation': '내지'}"
194496,노동,산업재해보상보험법 시행령,Articles 27 through 36 shall apply mutatis mut...,특수형태근로종사자에 대한 업무상의 재해의 인정 기준에 관하여는 제27조부터 제36조...,특수형태근로종사자에 대한 업무상의 재해의 인정 기준에 관하여는 제27조부터 제36조...,[],{}


## Grammer

### Grammer-Minor

In [268]:
def josa_mistranslation(df, ratio, seed):
    df_t = copy_df(df)
    js_mstr_index = random_index(df_t, ratio, seed)
    js_tokens = df_maad.loc[js_mstr_index].curHyp.apply(lambda x: kiwi.tokenize(x))

    jsd = {"가":"이", "을":"를", "로써":"으로써", "로서":"으로서", "로":"으로", "로부터":"으로부터", "란":"이란", "나":"이나", "와":"과"}
    keys = list(jsd.keys())
    for key in keys:
        jsd[jsd[key]] = key

    tags=["JKS", "JKC", "JKO", "JKB", "JX", "JC"]
    need_josa = None
    for i, tag in enumerate(tags):
        if i==0:
            need_josa = js_tokens.apply(lambda x: find_tokens(x, tag))
        else:
            need_josa += js_tokens.apply(lambda x: find_tokens(x, tag))

    drop_index = need_josa[need_josa.apply(len)==0].index
    js_mstr_index = list(set(js_mstr_index)-set(drop_index))
    need_josa.drop(index=drop_index, axis=0, inplace=True)
    np.random.seed(seed)
    trans_words = need_josa.apply(lambda x: x[np.random.choice(range(0,len(x)))])
    result = df_t.loc[js_mstr_index]

    for i in js_mstr_index:
        cur_hyp = result.loc[i, "curHyp"]
        word, start, end = trans_words[i]
        try:
            conv_word = jsd[word]
            conv_hyp = cur_hyp[:start]+conv_word+cur_hyp[end:]
            result.loc[i, "tagLog"]["grammer"] = conv_word
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "minor", start, conv_word, "grammer")
            dindex_dict["grammer"].append(i)
        except:
            pass
        try:
            o_index = result.loc[i, "tagLog"]["omission"]
            if o_index > start:
                if len(word) > len(conv_word):
                    result.loc[i, "tagLog"]["omission"] -= (len(word)-len(conv_word))
                elif len(word) < len(conv_word):
                    result.loc[i, "tagLog"]["omission"] += (len(conv_word)-len(word))
        except:
            pass
    return convert_df(df, result)

df_jmt = josa_mistranslation(df_can, 0.2, 1010)

In [269]:
df_jmt.loc[df_jmt.errors.apply(len)>0].shape

(10810, 7)

In [270]:
check_idx(df_jmt)

omission: 3190
addition: 7712
mis-translation: 0
over-translation: 0
under-translation: 0
grammer: 1936
############################################
omission:  3190
addition:  7712
mis-translation:  0
over-translation:  0
under-translation:  0
grammer:  1936


# 후처리

In [136]:
def convert_index(df_new):
    df = copy_df(df_new)
    for i in df.index:
        cur_hyp = df.loc[i, "curHyp"]
        for j, val in enumerate(df.loc[i, "errors"]):
            e_case = val["type"]
            if e_case=="omission":
                df.loc[i, "errors"][j]["start_idx"] = df.loc[i, "tagLog"][e_case]
            elif e_case=="grammer":
                continue
            else:
                idx = cur_hyp.find(df.loc[i, "tagLog"][e_case])
                df.loc[i, "errors"][j]["start_idx"] = idx
    return df

temp = convert_index(df_can)
temp.loc[[402202, 402265]]


,category,lawName,ENG,KOR,curHyp,errors,tagLog
402202,통화ㆍ국채ㆍ금융,보험업법 시행령,In other cases similar to subparagraphs 1 and ...,그 밖에 제1호 및 제2호에 준하는 경우로서 금융위원회가 정하여 고시하는 경우,그 밖에 제1호 및 제2호에 준하는 경우로서 금융정하여 차용만 고시하는 경우,"[{'severity': 'major', 'start_idx': 27, 'text'...","{'omission': 27, 'addition': '차용만'}"
402265,통화ㆍ국채ㆍ금융,보험업법 시행령,Where any contribution money has not been paid...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,"[{'severity': 'major', 'start_idx': 81, 'text'...","{'omission': 81, 'addition': '여신만', 'mis-trans..."


In [137]:
df_can.loc[[402202, 402265]]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
402202,통화ㆍ국채ㆍ금융,보험업법 시행령,In other cases similar to subparagraphs 1 and ...,그 밖에 제1호 및 제2호에 준하는 경우로서 금융위원회가 정하여 고시하는 경우,그 밖에 제1호 및 제2호에 준하는 경우로서 금융정하여 차용만 고시하는 경우,"[{'severity': 'major', 'start_idx': 27, 'text'...","{'omission': 27, 'addition': '차용만'}"
402265,통화ㆍ국채ㆍ금융,보험업법 시행령,Where any contribution money has not been paid...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,"[{'severity': 'major', 'start_idx': 77, 'text'...","{'omission': 81, 'addition': '여신만', 'mis-trans..."


In [140]:
df_can.loc[df_can.errors.apply(len)>0].shape

(10763, 7)

In [141]:
def make_dict(df):
    output_json = dict()
    output_json["data"] = []
    for i in df.index:
        base_form = {"category": None, "en": None, "errs": None, "hyp": None, "ko": None, "law_name": None}
        if len(df.loc[i, "errors"])==0:
            continue
        base_form["category"] = df.loc[i, "category"]
        base_form["en"] = df.loc[i, "ENG"]
        base_form["errs"] = df.loc[i, "errors"]
        base_form["hyp"] = df.loc[i, "curHyp"]
        base_form["ko"] = df.loc[i, "KOR"]
        base_form["law_name"] = df.loc[i, "lawName"]
        output_json["data"].append(base_form)
    return output_json

rrrr = make_dict(df_can)

In [142]:
with open(os.path.join(gdrive_path, "data_0825.json"), "w") as f:
  json.dump(rrrr, f, ensure_ascii=False, indent=4)